Data formatting for time series analysis of [German wind power generation](https://www.kaggle.com/jorgesandoval/wind-power-generation).

Changing files from column-based (with one row per date and one column per 15-minute interval) to row-based (such that each row indicates wind power generation at 15-minute intervals). Wind power generation in terawatt-hours (TWh).

In [1]:
import os
import numpy as np
import pandas as pd

In [28]:
path = '/Users/rohitsatishchandra/Desktop/MScA/Winter_2021/Time_Series/Assignments/final_project/data'
os.chdir(path)

In [29]:
os.getcwd()

'/Users/rohitsatishchandra/Desktop/MScA/Winter_2021/Time_Series/Assignments/final_project/data'

In [4]:
def file_transpose(input_file_path, output_file_path):
    """
    convert column-based time series matrix to row-based
    each row will indicate power generation at 15 minute interval
    """
    with open(input_file_path, "r") as input_file, open(output_file_path, "w") as output_file:
        # skip header
        next(input_file)
        # initiate line as first non-header row of input file
        line = input_file.readline().strip()
        # while line is not empty string (indicating end of file):
        while(line):
            # write a line to the output file with each value (except date) separated by new line
            output_file.write("\n".join(line.split(",")[1:]) + "\n")
            # read next line from raw input file
            line = input_file.readline().strip()

In [54]:
def file_transpose_daily(input_file_path, output_file_path):
    """
    convert column-based time series matrix to row-based
    each row will indicate daily power generation
    """
    with open(input_file_path, "r") as input_file, open(output_file_path, "w") as output_file:
        # skip header
        next(input_file)
        # initiate line as first non-header row of input file
        line = input_file.readline().strip()
        # while line is not empty string (indicating end of file):
        while(line):
            # sum values in each line to get daily production amount
            daily_sum = sum(map(float, line.split(",")[1:]))
            # write a line to the output file with each value (except date) separated by new line
            output_file.write(f"{daily_sum}\n")
            # read next line from raw input file
            line = input_file.readline().strip()

In [6]:
def hourly_transform(col, df, output_path, start_time='08/23/2019 00:00:00', end_time='09/22/2020 23:00:00'):
    """
    produce dataframe of hourly total power generation 
    and mean of 15-minute interval power measures per hour
    write dataframe to csv
    """
    hourly_total_series = col.groupby(col.index // 4).sum()
    hourly_mean_series = col.groupby(col.index // 4).mean()
    df['time'] = pd.date_range(start=start_time, end=end_time, freq='H')
    df.to_csv(output_path, index=False)
    return df

In [7]:
input_file_paths = ['./Amprion.csv', './TenneTTSO.csv', './TransnetBW.csv']
output_file_paths = [('./Amprion_out.csv', './Amprion_daily.csv', './Amprion_hourly.csv'),
                     ('./TenneTTSO_out.csv', './TenneTTSO_daily.csv', './TenneTTSO_hourly.csv' ),
                     ('./TransnetBW_out.csv', './TransnetBW_daily.csv', './TransnetBW_hourly.csv')]

paths = list(zip(input_file_paths, output_file_paths))
print(paths)

[('./Amprion.csv', ('./Amprion_out.csv', './Amprion_daily.csv', './Amprion_hourly.csv')), ('./TenneTTSO.csv', ('./TenneTTSO_out.csv', './TenneTTSO_daily.csv', './TenneTTSO_hourly.csv')), ('./TransnetBW.csv', ('./TransnetBW_out.csv', './TransnetBW_daily.csv', './TransnetBW_hourly.csv'))]


In [8]:
paths[0]

('./Amprion.csv',
 ('./Amprion_out.csv', './Amprion_daily.csv', './Amprion_hourly.csv'))

In [9]:
paths[0][1][2]

'./Amprion_hourly.csv'

In [11]:
for path in paths:
    file_transpose(path[0], path[1][0])
    print("Created column-based CSV.")
    file_transpose_daily(path[0], path[1][1])
    print("Created daily CSV.")
    #power_df = pd.read_csv(path[1][0], header=None, names=['power_twh'])
    #hourly_transform(power_df['power_twh'], power_df, output_path=path[1][2], start_time='08/23/2019 00:00:00', end_time='09/22/2020 23:00:00')
    #print("Created hourly CSV.")

Created column-based CSV.
Created daily CSV.
Created column-based CSV.


ValueError: invalid literal for int() with base 10: '9.68'

## amprion

In [15]:
amprion_daily = pd.read_csv('Amprion_daily.csv', header=None, names=['daily_power_twh'])
amprion_daily.shape

(397, 1)

In [16]:
amprion_daily.head()

,daily_power_twh
0,2262
1,3157
2,1426
3,1195
4,1490


In [17]:
amprion_qtr_hour_pwr = pd.read_csv('./Amprion_out.csv', header=None, names=['power_twh'])

In [18]:
amprion_qtr_hour_pwr.shape

(38112, 1)

In [19]:
amprion_qtr_hour_pwr.head()

,power_twh
0,5
1,5
2,6
3,7
4,6


In [20]:
amprion_hourly_total = qtr_hour_pwr['power_twh'].groupby(qtr_hour_pwr['power_twh'].index // 4).sum()

In [21]:
amprion_hourly_mean = qtr_hour_pwr['power_twh'].groupby(qtr_hour_pwr['power_twh'].index // 4).mean()

In [22]:
amprion_hourly = pd.DataFrame(
    {'hourly_total': amprion_hourly_total,
     'hourly_mean': amprion_hourly_mean}
)

In [23]:
amprion_hourly['time'] = pd.date_range(start='08/23/2019 00:00:00', end='09/22/2020 23:00:00', freq='H')

In [24]:
amprion_hourly = amprion_hourly[['time', 'hourly_total', 'hourly_mean']]

In [25]:
amprion_hourly.shape

(9528, 3)

In [26]:
amprion_hourly.head()

,time,hourly_total,hourly_mean
0,2019-08-23 00:00:00,23,5.75
1,2019-08-23 01:00:00,21,5.25
2,2019-08-23 02:00:00,18,4.50
3,2019-08-23 03:00:00,17,4.25
4,2019-08-23 04:00:00,24,6.00


In [30]:
amprion_hourly.to_csv('./Amprion_hourly.csv', index=False)

## TenneTTSO

In [59]:
file_transpose_daily('./TenneTTSO.csv', './TenneTTSO_daily.csv')

In [60]:
tennet_daily = pd.read_csv('TenneTTSO_daily.csv', header=None, names=['daily_power_twh'])
tennet_daily.shape

(397, 1)

In [61]:
tennet_daily.head()

,daily_power_twh
0,2655.76
1,6567.33
2,3501.63
3,3402.77
4,3346.76


In [62]:
file_transpose('./TenneTTSO.csv', './TenneTTSO_out.csv')

In [63]:
tennet_qtr_hour_pwr = pd.read_csv('./TenneTTSO_out.csv', header=None, names=['power_twh'])

In [64]:
tennet_qtr_hour_pwr.shape

(38112, 1)

In [65]:
tennet_qtr_hour_pwr.head()

,power_twh
0,9.68
1,10.16
2,10.94
3,11.39
4,12.09


In [66]:
tennet_hourly_total = tennet_qtr_hour_pwr['power_twh'].groupby(tennet_qtr_hour_pwr['power_twh'].index // 4).sum()

In [67]:
tennet_hourly_mean = tennet_qtr_hour_pwr['power_twh'].groupby(tennet_qtr_hour_pwr['power_twh'].index // 4).mean()

In [68]:
tennet_hourly = pd.DataFrame(
    {'hourly_total': tennet_hourly_total,
     'hourly_mean': tennet_hourly_mean}
)

In [69]:
tennet_hourly['time'] = pd.date_range(start='08/23/2019 00:00:00', end='09/22/2020 23:00:00', freq='H')

In [70]:
tennet_hourly = tennet_hourly[['time', 'hourly_total', 'hourly_mean']]

In [71]:
tennet_hourly.shape

(9528, 3)

In [72]:
tennet_hourly.head()

,time,hourly_total,hourly_mean
0,2019-08-23 00:00:00,42.17,10.5425
1,2019-08-23 01:00:00,52.09,13.0225
2,2019-08-23 02:00:00,60.98,15.2450
3,2019-08-23 03:00:00,72.51,18.1275
4,2019-08-23 04:00:00,84.37,21.0925


In [73]:
tennet_hourly.to_csv('./TenneTTSO_hourly.csv', index=False)

## TransnetBW

In [55]:
file_transpose_daily('./TransnetBW.csv', './TransnetBW_daily.csv')

In [56]:
transnet_daily = pd.read_csv('TransnetBW_daily.csv', header=None, names=['daily_power_twh'])
transnet_daily.shape

(397, 1)

In [58]:
transnet_daily.head()

,daily_power_twh
0,1291.94
1,660.47
2,76.63
3,174.12
4,279.39


In [33]:
file_transpose('./TransnetBW.csv', './TransnetBW_out.csv')

In [34]:
transnet_qtr_hour_pwr = pd.read_csv('./TransnetBW_out.csv', header=None, names=['power_twh'])

In [35]:
transnet_qtr_hour_pwr.shape

(38112, 1)

In [36]:
transnet_qtr_hour_pwr.head()

,power_twh
0,1.44
1,1.38
2,1.38
3,1.50
4,1.77


In [45]:
transnet_hourly_total = transnet_qtr_hour_pwr['power_twh'].groupby(transnet_qtr_hour_pwr['power_twh'].index // 4).sum()

In [46]:
transnet_hourly_mean = transnet_qtr_hour_pwr['power_twh'].groupby(transnet_qtr_hour_pwr['power_twh'].index // 4).mean()

In [47]:
transnet_hourly = pd.DataFrame(
    {'hourly_total': transnet_hourly_total,
     'hourly_mean': transnet_hourly_mean}
)

In [48]:
transnet_hourly['time'] = pd.date_range(start='08/23/2019 00:00:00', end='09/22/2020 23:00:00', freq='H')

In [49]:
transnet_hourly = transnet_hourly[['time', 'hourly_total', 'hourly_mean']]

In [50]:
transnet_hourly.shape

(9528, 3)

In [51]:
transnet_hourly.head()

,time,hourly_total,hourly_mean
0,2019-08-23 00:00:00,5.70,1.4250
1,2019-08-23 01:00:00,8.06,2.0150
2,2019-08-23 02:00:00,13.28,3.3200
3,2019-08-23 03:00:00,25.96,6.4900
4,2019-08-23 04:00:00,44.73,11.1825


In [52]:
transnet_hourly.to_csv('./TransnetBW_hourly.csv', index=False)

### 50Hertz

In [18]:
hertz_daily_output_path = "./50Hertz_daily_out.csv"

file_transpose_daily(input_file_path=input_file_path, output_file_path=hertz_daily_output_path)

In [22]:
pd.read_csv(hertz_daily_output_path, header=None, names=['daily_power_twh'])

,daily_power_twh
0,2949
1,5944
2,7359
3,3880
4,4460
...,...
392,4619
393,7521
394,6755
395,3473


In [74]:
hertz_qtr_hour_pwr = pd.read_csv('./50Hertz_out.csv', header=None, names=['power_twh'])

In [75]:
hertz_qtr_hour_pwr.head(4)

,power_twh
0,74
1,73
2,63
3,55


In [76]:
hertz_hourly_total = hertz_qtr_hour_pwr['power_twh'].groupby(qtr_hour_pwr['power_twh'].index // 4).sum()

In [77]:
hertz_hourly_mean = hertz_qtr_hour_pwr['power_twh'].groupby(qtr_hour_pwr['power_twh'].index // 4).mean()

In [79]:
hertz_hourly = pd.DataFrame(
    {'hourly_total': hertz_hourly_total,
     'hourly_mean': hertz_hourly_mean}
)

In [80]:
hertz_hourly['time'] = pd.date_range(start='08/23/2019 00:00:00', end='09/22/2020 23:00:00', freq='H')

In [81]:
hertz_hourly = hertz_hourly[['time', 'hourly_total', 'hourly_mean']]

In [82]:
hertz_hourly.shape

(9528, 3)

In [83]:
hertz_hourly.head()

,time,hourly_total,hourly_mean
0,2019-08-23 00:00:00,265,66.25
1,2019-08-23 01:00:00,210,52.50
2,2019-08-23 02:00:00,157,39.25
3,2019-08-23 03:00:00,122,30.50
4,2019-08-23 04:00:00,113,28.25


In [68]:
hertz_hourly.to_csv(path+'/50Hertz_hourly.csv', index=False)